In [ ]:
input_test_p1 = """RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE"""
input_test_a = """AAAA
BBCD
BBCC
EEEC"""
input_test_b = """EEEEE
EXXXX
EEEEE
EXXXX
EEEEE"""
input_test_c = """AAAAAA
AAABBA
AAABBA
ABBAAA
ABBAAA
AAAAAA"""
input_test_d = """OOOOO
OXOXO
OOOOO
OXOXO
OOOOO"""
output_test_p1 = 1930
output_test_p2 = 1206
input_output_p2_dict = {input_test_a : 80, input_test_b : 236, input_test_c : 368, input_test_d : 436}

In [ ]:
import numpy as np

In [ ]:
udlr = [np.array((-1,0)), np.array((1,0)), np.array((0,-1)), np.array((0,1))]

#From https://stackoverflow.com/questions/4808221/is-there-a-bounding-box-function-slice-with-non-zero-values-for-a-ndarray-in
def bbox2(img):
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    ymin, ymax = np.where(rows)[0][[0, -1]]
    xmin, xmax = np.where(cols)[0][[0, -1]]
    return img[ymin:ymax+1, xmin:xmax+1]

def parse_input(inp):
    return np.array([[c for c in line] for line in inp.split('\n')])

def count_corners(gx, group_id):
    corners = 0
    gxx = bbox2(gx == group_id) #exclude regions of all zeros to ease computational load
    for p in np.ndindex((gxx.shape[0] + 1, gxx.shape[1] + 1)):
        neighbors = []
        for o in map(np.array, [(0,0), (0,1), (1,0), (1,1)]):
            P = p - o
            if np.all(P >= (0,0)) and np.all(P < gxx.shape) and gxx[tuple(P)]: neighbors.append(True)
            else: neighbors.append(False)
        if neighbors in [[True, False, False, True], [False, True, True, False]]: corners += 2
        if sum(neighbors) in [1,3]: corners += 1
    return corners
            

def solve(inp, p2=False):
    g = parse_input(inp)
    gx = np.zeros(g.shape, dtype=int)
    group_id = 0
    total = 0
    group_id_to_area_dict = {}
    
    while np.any(gx == 0):
        seed_position = np.argwhere(gx == 0)[0]
        queue, seed_char, area, perimeter = [seed_position], g[tuple(seed_position)], 0, 0
        group_id += 1
        gx[tuple(seed_position)] = group_id
        while queue:
            p = queue.pop(0)
            area += 1
            #for every cardinal direction
            for d in udlr:
                P = p + d
                #if new position inside grid and same char
                if np.all(P >= (0,0)) and np.all(P < g.shape) and g[tuple(P)] == seed_char:
                    #if same char as seed position and not already visited, add to queue
                    if gx[tuple(P)] == 0:
                        queue.append(P)
                        gx[tuple(P)] = group_id
                #otherwise, increase perimeter
                else: perimeter += 1
        total += area * perimeter
        group_id_to_area_dict[group_id] = area
    if not p2: return total
    else:
        return sum([count_corners(gx, group_id) * group_id_to_area_dict[group_id] for group_id in group_id_to_area_dict])

In [ ]:
assert solve(input_test_p1) == output_test_p1
solve(input_full)

In [ ]:
for inp,outp in input_output_p2_dict.items(): assert solve(inp, True) == outp
assert solve(input_test_p1, True) == output_test_p2
solve(input_full, True)

In [ ]:
input_full = """CCKKKKKKKLLLLLLLLLLLLLLLLLSQQQQQQQQQQQQQQQQQQQQQQQQRRQQQQQQQQQQGGGGGGIIIIIIIPPPPPPPPPNNNNNNNNNNIIIIIIIIRRRRRRRRRRRUUUUUUUUUUTTTTTTNTTTTTTTTT
KKKKKKKLLLLLLLLLLLLLLLLLLLLQQQQQQQQQQQQQQQQQQQQQQQRRRRRRQQQQQQQGGGGGGGIIIIIPPPPPPPPPPNNNNNNNNNNIIIIIIIIRRRRRRRRRRUUUUUUUUUUUTTTTTTTVTTTHHTTT
KKKKKKCQQQLLLLLLLLLLLLLLLTQQQQQJQQQQQQQQQQQQQQQQQQRRRRRRRQRQQQGGGGGGGGGIIIIIIPPPPPPPPPPPNNNNNNNNIIIIIRRRRRRRRRRRRRRUUUUUUTTTTTTTTTTTTTTHHWWT
QIIKKKKQQQLQLLLLLLLLLLLLLLQSSQJJQQQQQQQQQQQQQQQQRRRRRRRRRRRQGGGGGGGGQQZZIIIIIPPPPPPPPPPNNNNNNNNNIIIIIIRRRRRRRRRRRRRUUUUUUTKKTCTTTTVVTTWHHWWW
QQIKKKQQQQQQQLLLLLLLLLLLLLSSSJJJJQJQQJQQQQQQQQQQRRRRRRRRRRRRGGGGGGGGZZZZIIIPPPPPPPPPNNNNNNNNNNNNIIIIIIIRRRRRRRRRRRRUUUUUCCKCGCTTTTTTTWWHHWWW
QQQKKQQQQQQQQLLDDLLLLLLLLSSSJJJJJJJJJJQQQQQQQQQQQQRRRRRRRRRRGGGGGGGGGZZZIIIPPPPPPPPPPMNNNNNNNNNNIIIIIIIIRRRRRRRRRRRUUUURRCCCCCTTTTTWWWWWWWWW
QQQQQQQQQQQQLLLDDLLLLLSSLSSSJJJJJJJJJJQQQDDDDQQQQQQRRRRRRRRRGGGGGGGGGZZIIIPPPPPPPPPPPMMNNNNNNNNIIIIIIIIISSRRRRRRRRRRRRRRRCCCCCTTTTTWWWWWWWWW
QQQQQQQQQQQLLLLDOLLLXLSSSSSSJJJJJJJJJJJQQDDDMMMMMMMMMMRRRRRRGGGGGGGGGZZZJIPIPPPPPPPPHWMWNNNNNNIIIIIISSSIVVVRRRRRUFRQRRRRCCCCCCCTTTTWWWWWWWWW
QQQQQQQQQQQLLLLDDDDDDSSSSSSJJJJJJJJJJJQQQQQDMMMMMMMMMMRRRRRGGGGGGGGGGJJBJIIIIPPPPPPWHWWWWNNNNNIIISSSSSSYVVVRRRVRFFFFURRRZCCCCCCSTTTTWWDWDWWW
QQQQQQQQQQQQQDDDDDDDDSSSSSSSJJJJJJJJJJDQQDDDMMMMMMMMMMURRRRGGGGGGGGGGJJJJJIIIPPPPPWWWWWWZNNNIIIIIISSSVVVVVVVRVVFFFFFUSRRCCCCCCCSSWWWWWDDDDWW
PPQQQQQQQQKQDDDDDDDDDDDSSSSSJJJDJIMJJPDDDWDDMMMMMMMMMMURRRRGGGGGGGGGJJJJJJIIIPPPSPWWWWWWWZZZCCCIIIXSVVVVVVVVRVVVVFFSSSSRCCCCCCCSSWDDWWWDDDDW
PPQQTQQQQKKKDDDDDDDDDDDDDSSSJJMMMMMPPPPDDDDDMMMMMMMMMMRRRDRGGGGGGJJJJJJJJJIIIIPPWWWWWWWWWCZZCCCIIIXVVVVVVVVVVVVVVVFSSSRRCCCCCCSSSSDDDWWDDDWW
PPPQPQQNQKKKKKKDDDDDDDDDURSSSMMMMMMMMPPPPDDDMMMMMMMMMMUDEDODDGGGGJJJJJJJJJJJBIPWWWWWWWWWCCZCCCCIIXXXXWVVVVVVVVVVVVVVYCCCCCCCCCSSSSDDDDDDDDDD
PPPPPQQPPPKKKKKDDDDDDUUUURRRSMMMMMMMMMPPDDDDMMMMMMMMMMDDDDDDDGGGGGJJJJJJJJJBBBDDDWWWWWWCCCCCCCXXXXWWWWVVVVVVVVJJJVVVYCCCCCCLSSSSSSVDDDDDDDDD
PPPPPPPPPJKKKKDDDDDUUUUURRRSSMMMMMMMLLPDDDDDDDDDDDDDDDDDDDDDDGGGGGGGJDJPPDDDDDDDDWWWWWWWWCCCCCCWWWWWWWWWVVVVVVJJJJWWJCCCCCLLSSSSSSSSSDDDDDDD
PPPPPPPPPPKKKKDDHDUUUUURRRRRRRMMMMMMLLLLLDDDDDDDDDDDDTDDDGGGGGGGGGGGGDPPDDDDDDDDDDDWWWWWWCCCCCWWWWWNWWWWWVVVVVJJJJJJJJCCCCLLSSSSSSSSBDDDDDDD
PPPPPPPPKKKKKKKKKKUUUUURRRRRRMMMMMLLLLLLLKKDDDDDDSSDDDDDDGGGGGGGGGGDDDDPPDDDDDDDDDDWWWGGGGGGGGGGWWWWWWWWWWVVVVVJJJJJJJJCCCCCSSSSSSSSSSDDDDDK
PPPPFFPPPKKKKKKKKKUUKURRRRRRRMJMMMMLLLLLLKKDDDDDSSSSDBDDDHHHGGGGGGDDDDDDDDDDDDDDDDDDDWGGGGGGGGGGWWWWWWWWWVVVVVVJJJJJJJJJCCCCSSSSSSSSDDDDDDDD
PPFPFFPPPKKKKKKKKKKKKULRRRMMMMMMMMMLLLLLLLDDDDSSSSSSUBBBDDHHHGGHHGHDDDDDDDDDDDDDDDDDDDGGGGGGGGGGWWWWWWWWWWWVVVVJCJJJJJJJCHHSGSSSSSSSSDDDDDDD
EEFFFPPPKKKKKKKKKKKKKULLRRRMMMMMMMMLLLLLLLBBSSSSWSSSSSBBDHHHHHHHHHHHDDDDDDDDDDDDDDDDGGGGGGGGGGGGWWWWWIIIIVVVVVJJCJJJJJJHHHSSSSSSSSSSSDDDDDQQ
EEFFEPPKKKKKKKLLLKKKKULLRRRRMMMMMMMMLULLLLBBSSSSSSSSSSBCCCCCCCCCHHHDDDDDDDDDDDDDDDDDGGGGGGGGGGGGWWWWISIIIIIVVVVVVNNJJJJJJSSSSSSSSSSSSSDDDDDQ
EEEEEKKKKKKKKKLLLKLLLLLRRRRRRMMMLMMMLRRRLBBBBSSSSSSSSSSCCCCCCCCCHHHDDDDDDDDDDDDDDDDDGGGGGGGGGGGGWWWRIIIIIIIIVVVVVNJJJJJJSSSSSSSSSSSSSSDDDDDQ
EEEEEEKKKKKGKKLLLLLLLLLLLRRRRRRRLLLRRRRRLBBBSSSSSSSSSSSCCCCCCCCCCHHDDDDFDDDDDDDDDDDDGGGGGGGGGGGGWWRGGGGGGGIAVVVVJJJJJJJJSSSSSSSSSSSSSSDDDDDD
EEEEFGGGGGKGGGLLLLLLLLLLLRRRRRRRLLLRRRRRBBBBBSSSSSSSSSCCCCCCCCCCCHDDDDDFDLDDDDDDDDDDGGGGGGGGGGGGGGGGGGGGGGIIIVJJJJJJJJIIISIISSSSRSSSSSDDDDKK
ECCCCGCGGGGGGGLLLLLLLLLMRRRRRRRRLLLLLRRCBBBCSSSSSSSSSSCCCCCCCCCCCHDDDDDFDLDDDDDDDDDDGGGGGGGGGGGGGGGGGGGGGGIVVVVVJJJJJJIIIIIRESSSRRSRRSRRKDKK
ECCCCGGGGGGGGLLLLLLLLLLLRRRRRRRRRLLLCCCCBBCCCDSSSSSSSSCCCCCCCCCCCDDDDDDDDLDDDDDDDDCCGGGGGGGGGGGGGGGGGGGGGGIVVVVXXXJJXXXIIRRRRRRSRRRRRRRRKKKK
CCCCUGGGGGGGGLULLLLLLSSSRRRRRRRRLLLLLCCCCCCCCSSSSSSSSSCCCCCCCCCIIIDDIDDDDLLLLDDDDDCCGGGGGGGGCCCCCCRGGGGGGGFVVVXXXJJJXXXIIIRRRRRRRRRRRRRRRKKK
CCCCCCGMGGGGGLLLLLLLLSSSSSSRRRRRRLLLLLCCCCCCCSBSSSSSSSCCCCCCCCCQIIIDIDDRDDLLLDDDDDCCGGGGGGGGCCCCCRRGGGGGGGFFFVVXXXXXXXXXIRRRRRRRRRRRRRRRRRKK
CCCCCCCCGGGGLLLLLLLLLLLSWSSRRLRLLLLLPLLCCCCCCSSSSSSSSSCCCCCCCCCQQIIIIIDDDDLLLLLDDLCCCCCGGGGGPCCCCRRGGGGGGGFFFFVXXXXXXXEXXRRRRRRSRRRRRRRRRRRK
CCCCCCCCGGGGGGGLLLLLLSSSWWRRLLLLLLLLPCCCCCCCCSNSSSSSSSCCCCCCCCCQIIIIIIDKKLLLLLLLDLLLCCCGGGGGPPPPRRNGGGGGGGNFFXXXXXXXXXXXDRRRRSSSSSRSSRRRKKKK
CCCCCCCCCGGGGGLLLLLLSSSSWWRRLLLLLPPPPPCCCCCCCCNNNSSSMMCCCCCCCCCIIIIIIKKKKLLLLLLLLLLLCCCGGGGGPPPPRRNNNNNNNNNFFXXXXXXXXXIIGGGGGSSSSRRSSRRSSSKK
CCHCCCTCMMGGGSSSSLSSSSSSWWTRLLLLYYPPPPPCNCCCCNNNSSSSSMCCCCCCCCCIIIIIKKKKKLLLLLLLLLVVVPPPPPPPPPPNNNNNNNNNNNNFFXFXXXXXXXXIIGGGCSSSSSRSSSSSSKKK
CCCCCWTTGGSSSSSSSSSSSSWWWWLLLYYYYYPPPPMNNNCCCNNNSSSSMMCCCCCCCCCIIIIIKKKKKKKLLLLLLLVVVVPPPPPPPPPNNNNNNNNNNNNNFFFXGXEXXXIIIIGGCSSSSSSSSSSSKKKK
CIIIWWTTTGGGGGGGGSSSSSSSWSSLLYYYYYYPPPNNNNNNNNNNNSMMMMMMMMMMMIIIIIKKKKKKKKKKKILLLLVVVVPPPPPPPPPPNNNNNNNNNNNNFFFFFEEEXXIIIIGGGSSSSSSSSSKKKKKK
IIIWWTTTGGGGGGGGGGGSSSSSSSSSYYYYYYYYYYNNNNNNNNNNNSMMMMMMMMMIIIIIIIIKKKKKKKKKKKLLLLVVVVPPPPPPPPPNNNNNNNNNNNNNFFFFFEEEXXXXIGGGGSSSSSSSSSKKKKKK
IIIIITTTTGGGGGGGGGSSSSSSYYYYYYYYYYYYNNNNNNNNNNNNNNAMMMMMMMMIIIIIIIIKKKKKKKKKKKLBBLLVVPPPPPPPPPPNNNNNNNNNNNNNFFFFFFEEXXXXIGGGGSSSSSEEKKKKKKKK
IIITTTTTTGGGGGGGGGGGSSSSSYYYYYYYYYYYVNNNNNNNNNNNNNNUMMMIIMIIIIIIIIKKKKKKKKKKKKBBBBLLLLPPPPPPPPPPNNNNNNNNNBNBFJFFEFEEXXXXXGGGGGSSEEEEKKEKKKKK
IIITTTTTTGGGGGGGGGGGSRSSSSYYYYYYYYYYYYNXNNNNNNNNNNNNCMMIIIIYYIIIIIKKKKKKKKKGGKBBBBLLLLPPPPPLPPPPNNNNNNNNBBDDDJFFEEEXXXXXJJGGJGSSREEEEEEKCKKK
IIITTTTTTGGGGGGGGGGGGRRRYYYYYYYYYYYYYYXXNWWNNNNNNNNNCMMXXIYYYIYIIIIKKKKKKKKKKKBBBBLLLLLLPPPLPPPPWWNNNINNNDDDDDEEEEEXXXXXJJJJJJSHEEEEEEEECKKC
IXIIIIITPPPGGGGGGGGGGRRRRYYYYYYYYYYYYYYYWWWNNNNNNNNNNMXXXXXYYYYYYIIKKKKKKKKKRKBBBBBXXXLPPPPLPPPPWWANIIIIIDDDDDDDEEFXXXXXXXEEEESEEEEEEEECCKCC
XXXXXIIIPPGGGGGGGGGGGRRRRRYYYYYYYYYYYYYYYWNNNNNNNNXXXXXXXXYYYYYYYYYKKKKKKKKKKBBBBBBXXXLLLLLPPPPWWWAIIIIIIDDEEDEEEFFFXXXXXEEEEEEEEEEEECCCCCCC
XXXXIIIIIPGGGGGGGGRRRRRRRYYYYYYKYYYYKYXXXJWNNNNNNXXXXXXXXXYYYYYYYYYAAKKKKKKKKBBBBBBXXXXXXXLLVPWWWWAAAIIIDDDDEEEFEFFFXXXEEEEEEEEEEEEEEECCCCCC
XXXXXIIIPPGGGGGGGGRRRRRRRYYYYYYKYYKKKKXXXJJSNNNNXXXXXXXXXYYYYYYYYYAAKKKKKKKKBBBBBBBBBXXXXXLLVPAAWWAAAAIIDDDDJEEFFFFFXXXEEEEEEEEEEEEEECCCCCCC
XXXXSSSSWWWWPGGNNNRRRRRRRRYYYYKKKKKKKXXXXJJJJNNNXXXXXXXXXYYYYYYYYYYAAUUKKKKKBBBBBBBBBXXXXXXAAJAAAWAAIITTTTIIJEFFFFFXXXXEEEEEEEEEEEEECCCCCCCC
XXXXXWWWWWWWGGNNNNRRRRRRRRYYYKKKKKKKKKXXXXXJJJNNXXXXXXXXXYYYYYYYYYAAAAKKLKKBBBBBBBBBBXXXXXXAAAAAAAAAAATTTTIJJJXFFFFXFXXFEEEEEEEEEEECCCCCCCCC
XXXXTWWWWWWWWGWNNRRRWRRRRRMYYJJKKKKKKXXXXJJJJJJNXXXXXXXXYYYYYYYYYYAAAAAALLLLBBBBBBBBXXXXXXAAAAATTTTTTTTTTTIJJJFFFFFFFFFFFEEEEEEEEEEECCCCCCCC
HTTTTJWWWWWWWGWRRRRWWRRRRRYYYJJKKKKKKKXJJJJJJJJJJXXXXXXXYYYYYYYYYAAAAAALLLLLBWBBBBBBXZZZXXAAAAATTTTTTTTTTTJJJJJFFFFFFFFFFEEEEEEEEEEECCCCCCCC
TTTTWWWWWWWWWWWWWWWWRRRRRYYJJJWWKKKKKKKKJJJJJJJJJXXXXXXXUYYYYYYAAAAAAAALLLLLLBBBBBBBBZZZAAATTTTTTTTTTTTTTTTJJFFFFFFFFFFEEEEEEEEEEEEOCCCCCCCC
VVTTWWWWWWWWWWWWWWYYYYURRWYJWWWWKKKKKMKJJJJJJJJJXXXXXXXXUYYYYYYAAAAAAAALLLLLLBBBBBBZZZZZAAATTTTTTTTTTTTTTTTJJJFFFFFFFFFEEEQEEEEEEEEOCPCCCCCC
VVTWWWWWWWWWWWWWWWYYYYYPWWWWWWWWKKKKKMMJJJJJJJJJEEXJJNVXVVSVVVVVVAAAAAALLLLLLLLLSVSZZZZAAAATTTTTTTTTTTTTTTTJJJFFFFFFFFFFEEEEEEEEOOOOCCCCCCCC
VVVWWWWWWWWWWWYYYYYYYYYYYWWWWWWWWKKKKMMMJJJJJJJEEENNNNNNVVVVVCVVVVAFAAFLLLLLLLLXSSSZZIZAAAATTTTTTTTTTTTTTTJJJJFFFFFFFFFEEEEBEEEEEZZZZCCCCCCC
VVVWWWWWWWIWWWCYYYYYYYYYYWWWWWWWWWKKKMMMMMJJJJJJENNNNNNNVVVVVVVVVVFFFFFFLLLLXXXXDSSZIIAAAAATTTTTTTTTTTTTJJJJJJJKFFFFFFEEEEZBZEDZZZZZZCCCCCCJ
VVWWWWWWWWWWWWYRRYYYYYYYYWWWWWWWWWWKPPPPPMMMMMJJJNNNNNNVVVVOVVVVVVFFFFFFFLLXXXVXSSSIIIAAAAATTTTTTTTTTTTTJJJJJJJFFFFSTTEEEEZZZZZZZZZZZCCCCCJJ
DDDWWWWWWWWWWYYYYYYYYYYYWWWWWWWWWWWPPPPPPPMMMMJJJZNNNNNVVVVVVVVVVFFFFFFFFFXXXXXXSSSIIIAAAAATTTTTTTTTTTTTJJJJJJFFFYFFTTTTTEZZZZZZZZZZZZZCCCJJ
DDBWWWWWWWWWWYYYYYYYYYYYWWWWWWWWWWWPPPPPPPPMMMMMJZZNNNNVVVVVVVVVVFNFFFFFFFXXXXXXSSIIIIAIAAATTTTTTTTTTTTTJJJJJJJJFYYTTTTTTEZZZZZZZZZZZZZYCCCC
DDDDWWWWWWWWWYYYYYYYYYYYYWWWWWWWWWPPPPPPPPPPMMIIIZZNNNNNNNVVVVVVVVNFFFFFFFXXXXXXSSIIIIIIIAATTTTTTJJJJJJJJJJJUUUYYYJTTTTTTZZZZZZZZZZZZZZYYCCR
DDDWWWWWWWWWWYYYYYYYYYYYYWWWWWWWWPPPPPPPPPPPIIUIIIZIINNNNVVVVVVFFNNFFFFFFFFXXXXXSSSTTIIXSAATTTTTTJJJJJJJJJJJUUUPJJJJTTTCCZZZZZZZZZZZZZZYYWCR
DDDDWWWWVWWWWYIGYYYYYYYYYWWWWWWWLLPPPPPPPPPPIIIWWWWWIINNNNNNVVVFSFNFNFFFFFFXXXXXXXXXXXHXSAAAHZZZZZHHZZJJJJJVUUUPJJJJTTTTTFFFFZZZZZZZZZWWYWWR
VDWWWWWVVVWGWYGGYYYYYYYYYWWWWWWWPPPPPPPPPPPIIIIWWWWWINNNNNNNNNNFFFNNNFFFFFXXXXXXXXXXXHHXSAXHHHZZHHHHZZJJJJJVUUUPJJJTTTTTTTFFFFFZZZZWWZWWWWWW
VWWVWWVVVVWGGGGGYYYYYVPYYYWWWLWLLPPPPPPPYPDIIIIWWWWWINNRNNNNNNFFFFFNNFFKFXXXXXXXXXXXXHXXXXXXHHHHHHHZZZJJJJGVVUPPJPGTTTTTCTFFFFFZWWWWWWWWWWEE
VVVVWUVVVVVGGGGGGGGZZPPPPYWWWLLLPPPPPPPPPIIIIIIWWWWWRRRRNNNNNNFFFFFFFFFFFXXXXXXXXXXXXHHXXXXXHHHHHHHHHHJJJJGGPPPPPPPBGGTTTTFFOFFZWWWWWWWWWWEE
VVVVWVVVVVVFGGGGGGPPPPPPPPPPWLLLPPPPPPPPPPIIIIIWWWWWNNNNNNNNNFJFFFFFFFFFFFXQXXXXXXXXXHHXXXXXHHHHHHHHHHGGGGGGPGGPGBBBGGGGJOOFOFOOWWWWWWWWWWWW
VVVVWVVVVVVVVVGGGXXXPPPPPPPPILLPPPPPPPLPYYIIIIIWWWWWNNNNNNNNNFFFFFFFFFFFFFQQHHXHHXXXDFHHHHHHHHHHHHHHHHGGGGGGGGGGGGBGGGGGOOOOOOOHWWWWWWWWWWWW
VVVVVVVVVVVVVVVVVXXXXXXPPPPPILLPPPPPPPPYYXIIIIIWWWWWWWWWWWNNNNFFFFFFFFFFFFFQHHHHHXHFFFFFFHHHHHHHHHHHHHHGGGGGGGGGEEGGGGGGOOOOOOOWWWWWWWWWWWWW
VVVVVVVVVVVVVVVVXXXXXXXXPPPPILIIIPIPLLPYXXIPIIIWWWWWWWWWWWNNNFFFFFFFFFFFFFYQHHHHHHHFFPFFFHHHHHHHHHHHHHHGGGGGGGGGGEEEGGGGGOOGOOWWWWWWWWWWWWWT
VVVVVVVVVVVVVVVVVVXXXXXPPPPPIIIIIIIIIIIIPXXXIIIWWWWWWWWWWWNNNFLFPPFFFFFFFFYQHHHHHHHFFFFFFHHHHHHHHHHHHHHGGGGGGGGEEEEGGGGGGGGGGGGWWWWWWWWWWWTT
VVVVVVVVVVVVVVVVVVXXXXXXXPIIIIIIIIIIIIIIPXXXXXIIWWWWWWWWWWNNNNNNPPPFFFFFFFYHHHHHHFFFFFFFFFFFHFFFHHHHHHHGGGGGGGGGEEEGGGGGGGGGGBBYWWWTWWWWWTTT
VVVVVVVVVVVVVVVVVXXXXXXXXPPIIIIIIIIIIIIKXXXXXXXIWWWWWWWWWWNNNNPPPPPFFFFFHHYYHHHHFFFFFFFFFFFFFFFZHHHHHHHHGGGGGGGGEPGDDGGGBBBGBBBYYYBTWWWWTTTT
VVVVVVVVVVVVVVVVVVXXSSSSSYSSIIIIIIIIIXKKXXXXXXXIWWWWWWWWWWNNPPPPPPFFFFFHHHYYHHHHFFFFFFFFFFFFFFPZZHZHHHHHHHGGGKKGPPGGGGGGBBBGBBBBBBBTWWWTTTTT
VVVVVVVVVVVVVVVVVXXXSSSSSSSSFFFIIIIIIXXKXXXXXXXXWWWWWWWWWWNNNPPPPPPPFFHHHHHHHHHHFFFFFFFFAFFFFPZZZZZHHHHHHHGGKKSSPPPOOGGGBBBBBBBHBBBBBDBBTTTT
VVVVVWVVVVVVVVVVUSXSSSSSSMMMMFIIIIIIIXXXXXXXXXXXXXWWWWWNNNNNNNNPPPFFFFFFHHHHHHHHFFFFFFFAAAFFFZZZZZZZHHHHHHGGKKKPPOOOOGGBBBBBBBBHHBBBBBBTTTTT
VVVVVWWVVVVVVVVVUSSSSSSSSSMMMFFIIIIIIXXXXXXXXXXXXXWWWWWEMNNNNNNPPPPPFHFHHHHHHHHFFFFFFFFFFFFFFDZZZZZLHHHHHHHSKKKPPPPOOOGGLBBBBBZBBBBBBBTTTTTT
VWWVWWWWWVVVVVUUUUSSSSSSSSSMMFFIIIIIXXIXXXXXXQQQQXWWWWWEMIMNMWMPMMPPPHHHHHHHHHHHHFFFFFFFFFCCCZZZZZZLHIIHHHHSSKPPPPPOOOGOBBBBBBBBBBBBBBTTTTTT
VWWWWWWWWWWWVUUUUUSSSSSSSMMMPFFFIPIIIIIIXXXQQQQQQQWWWWWMMIMMMMMMMMMPPPHHHHHHHHHHQSFFFFFFFCCCCZZZZZZLHHHHHHHSSSSPPPOOOOOOBBBBBBBBBBBBBTTTTTTT
WWWWWWWWWWWWWUUUUUSSSSSSSSMMPFPPIPPIIIIIXXXQQQQQQQQQQQMMMMMMMMMMMMMMPPHHHHHHJJJJSSSSFFCCCCCZZZZZZLLLSSSHHHHSSSSSSOOOOOOOOOBOOSSBBBBBBTTTTTTT
WWWWWWWWWWWWWUUUUGCSSSSSSSMPPPPPPPPPIIIIQQXQQQQQQQQQQQQMMMMMMMMMMMMMPMMDHHHHJSSSSSSSSCCCLLLZZZZZZLLLLSSSSSSSSSSSSSOOOOOOOOOOSSSSSBBBBBTTTTTT
WWWWWWWWWWWWUUUUUGCSSSSSMMMPPPPPPPPPPIIIJQQQQQQQQQQQQQQQQMMMMMMMMMMMMMDDDHHDSSSSSSSSSCCLLLLZZZZZZLLLLSSSSSSSSSSSSSSOOOOOOOOOSSSSSSSBTBBTTTTT
WWUWWWWWWWWWUUUUUGSSMSSMMGMPPPPPPGEEEEEEJJQNQQQQQQTQQQQQQMMMMMMMMMMMMMMDDHHDSSSSSSSSSSSLLLLZZZZZZLLLLLSSSXSSSSSSSOOOOOOOOOOOSSSSSSSTTTTTTTTT
WWWWWWWWWWWWUUUUUGSSMMMMMGMPPPPPPPPEEEEEEJQQQQQQTQTTTTQYAAAAAAAMMMMMMMMDDDDDSSSSSSSSSSSSLLLLLZZLZLLLLLSSSSSSSSSSSOOOOOOOOOOSSSSSSSSTTTTTITII
WSSSSSWWWWWWWUGGGGSSSMMMMGGPPPPPPPEEEEEEEEQQQRRTTQTTTLJYAAAAAAAAAAMMMMMDDDDDDDSSSSSSSSSSLLLLLLLLLLLLLLSSLSSSSSSSOOOOOOOOOOOOOOSSSTTTTTTIIIII
TTSSSSSSWWWWTGGGGGGGGGMMMGGDDDPPPPEEEEEEEEQQQTTTTTTTTLJJAAAAAAAAAAMMMDMMDDDDDDSSSSSSSSSLLLLLLLLLLLLLLLLLLLSSSSSOOOOOOOOOOOOOSSSSSSSTTTIIIIII
SSSSSSSSSWWTTGGGGGGGGGGGMGGGGGPPPPEEEEEEEEERRTTTTTTTTJJJAAAAAAAAAAMDDDDDDDDDDDSSSSSSSSSSLSSLLLLLLJJZJLLLLLLCSCCCCCOOOEEEOOOSSSSSSSSSIIIIIIII
SSSSSSSSSSWWTGGGGGGGGGGGGGGGGGEEEEEEEEEEEERRRRTTTTTTTJJJAAAAAAAAAAMPPDDDDDDDDSSSSSSSSSSSSSJJJJJLLJJJJJBBBCCCCCCCCCCOOEEEEEEESSSSSSSSSSIIIIII
YSSSSSSSSSLWTTJGGGGGGGGGGGGGGGEEESEEEEEMERRTTTTTTTTTTJJJAAAAAAAAAAPPPDDDDDDDDSSSSSSSSSSSSAAJJJJJJJJJJBBBBCCCCCCCCCCEEEEEEKESSSPVVSSSIIIIIIII
YSSSSSSSSSJJTTJJJGGGGGGGGGGGGSSSSSEKMMMMMRRRTTTTTTTJJJJJAAAAAAAAAAPPPDDDDDDDSSSSSSSSSSSSSSAAJJJJJJJJJBBBBBCCCCCCCCCEEEEEEERRSSVVVVVVDDIIIIII
YSSSSYYSSWJJJJJJJJJGGGGGGGGXGSSSSUUKKMMMRRRRTTTTTTTJJAAAAAAAAAAAAARRPPDDDDPPXSSSSSSSSSSSSSAAAJJJJJJJJBBBBBBBCCCCDDEEEEEEEEERSVVVVVVVVVIIIIII
YYSSSYYYYJJJJJJJJJGGGGGGGGGGSSSSUUUKMMMMMMRRRTTTTTTJJAAAAAAAAAAAAAPPPPPPDDPPXXSSSSSAAAAAAAAAJJJJJJJJJJJJJJBBCCCCDZZZEEEEEEEENVVVVVVVVVIIIIII
YYSYYYYJJJJJJJJJJGGGGGGGGGGSSSUSUUMMMMMMMMRTTTTTTJTJJAAAAAAAAAAAAAPPPPPPPPPPPXSXSSSAAAAAAAAAJJJJJJJJJJJJJJBBBBBBZZZZEEEEEEEEVVVVVVVVVVIIFIII
YYYYYYYJJJJJJJJJJJJJGGGGGGGSSSUSUUMMMMMMMMMMBBTTJJJJJAAAAAAAYYRRPPPPPPPPPPPPPXXXSXXKAAAAAAAAJJKJJJJJJJJJJJBBBZZZZZZZEEEEEERRRVVVVVVVVVVVFFFF
YYYYYJJJJJJJJJJJFFJJJGGGGGGUUUUUUMMMMMMMMMMMMBTTTJJJJAAAAAAAYYRPPPPPPPPPPPPPPXXXXXXKKKKKAAAJJJJJJJJJJJJJBBBBBZZZZZZZEEEEEEVVVVVVVVVVVVVVVFFZ
YYYYYJJJJJJJJJJJJJJJIIIGGGUUUUUUUMMMMMMMMMMMBBTTJJJJJAAAAAAAYYRPPPPPPPPPPPOOXXXXXXXKKKKKKAAZIIJJJJJJJJJJBBBBBBBZZZEEEEEEEEEVVVVVVVVVVVVVVVZZ
YYYYJJJJJJJJJJJJJJJJIITTGGUUUUUUUUMMMMMMMMMMMMJJJJJJJAAAAAAARRRPPPPPPPPPPPOOOXXXXXXKKKKKKKAZIIZJJJJJJJJJBBBBBBBBZZZEEEEEEVEVVVVVVVVVVVVVVFOZ
YYYYYJJJJJJJJJJJJJJJIIIIIIUUUUUUUUCCCMMMMMMMMMMMMJJJJJJYYYYYYRYYPPPPPPPOOOOOOXXXKKKKKKKKKKZZZZZJJJJJJJJJBBBBBBBBBSZUEEEEEVVVVVVVVVVVQOOVOOOZ
YYYYYJJJJJJJJJJJJJZZIIIIIIUUUUUUUUUUUMMMMMMMMMMMMJJJJJJJYYYYYYYYPPPPPPPOOOOOOOOOKKKKKKKKKZZZZZZJZZJJJJJJBBBBBBBBBBBUUUEEEEVVVVVVVVVVVOOOOOOZ
YYYYYJJJJJJJJJJJJMBIIIIIIIIUUUUUUUUUUMMMMMMMMMMJJJJJJJJJJYYYYYYYYYYPPPPOOOOOOOOOKKKKKKKKKZZZZZZZZZJJJJHJBBBBBBBBBBBUUUEUUUUVVVVVVVVVVOOOXXOZ
YYYYYJJJJJXXJJJJBBBDIIIIIIEUUUUUUUUUUMMMMMZKFFMMJJJJJJJJJYDYYJYYYYYYYPPOOOOOOOOOOOKKKKKKKZZZZZZZZZZJJJHHHHBBBBBBBBBUUUUUUVVVUVVVZVBBOOOOXXOO
YYYYYJYYJJHXHJJJBBBDIIIIEEEEEUUUUUUUUMFMMMKKKKCCCJJJJJJJJJDDYYYYYYYYPPPOOOOOOOOOOOKKKKKKKZZZZSZZZZZJJJHHHHNBBBBBBUUUUUUUUUUUUVJVVVOOOOOOOXOO
YYYYYYYYHHHHHHHBBBBIIIIIEEEEEUUUUUUNNMFMFFKLKKCCJJJJJJJJJJDDYYYYYYYYPPPOOOOOOOOOOKKKKKKKKKZZZZZZZZZJJIKHHHNNBBBBBUUUUUUUUUUUCSCMVEEOOOOOOOOO
YYYYYYYYYRXHXHBBBBBIIIIEEEEUUUUUUUUUUFFFFFFKKKKCJJJJJJJJJMYYYYYYYYYYYOOOYYOOOOOOOOKKKKKKKKZZZZZZZZZZKKKHHNNNNNBBBUUUUUURRUCCCCCCEEEOOOOOOOOO
YYYYYYYYYRHHHBBBBBBBBBBBBBBKUUUUUUUUUUUFFFFKKKKKKJUCCCCCCYYYYYYYYYYYYYYOOOOOOOOOOOOKKKKKKKKKZZZZZZZZKHHHNNNNNNBBBUUUURRRQCCCCCCCEEOOOOOOOOOO
YYYYYYYYYRBBBBBBBBBBBBBBBBBKUUUUUUUUUUUUKFKKKKKKKUUCECCCCDYYYYYYYYYYYYYAOOOOOOOSOOKKKQQQKKKKKZZZZZKKKHHHHHNNNNNNNNRRRRRRQCCCCCCCEELOOOOOOOOO
YYYYYYYYRRRBBBBBBBBBBBBBBBSUUUUUUUUUUUUAKKKKKKKCKCCCCCCCCDDYYYYYYYYYYYYAAOOOOOOOHHKKKQQQKKKKZZZRZZKKKHHKKHNNNUNNNRRRRRRRCCCCCCCTLLLOOOOOOOOO
YYYYKUUUUUBBBBBBBBBBBBBBBBSSUUTUUUUUUUAAKAKKKKKCCCCCCCCCCCCWYYYYYYYYYYYYYHOOBOOHHHQQQQQQQQQQBBBBZZZKKKKKKHKKKKNNNRRRRRRRCCCCCCCCLLVOOOOOOOOO
YYYYKKUUUUUBBUUUBBBBBBBBBBSSSUUSUUUUUUAAAAKKKKKKKCCCCCCCCCCCCCCYYYYYYHHHHHHOOOHHHHQAQQQQQQQQBBBZZKKKKKKKKKKKKYYNNRRRRRRCCCCCCCCCLLLLOOOOOOOO
YYKKKKUUUUUUUUUBBBBBBBBBBBSSSUSSUUUSAAAAAKKKKKKKKCCCCCCCCCCCCCCYYYYYYYHHHHHOHHHHHQQQQQQQQQQQBBBBBKKKKKKKLKKKKYYYRRRRRRCCCCCCCCCCLCJJZZZODOOS
YYKKKKKUUUUUUUUBBBBBBBBBBBBBSSSSSUUSAQAAAKKKKKKKKKCCCCCCCCCCZZZYYYYYYHHHHHHHHHHHHHQQQQQQQQQQBBBIBKKKKKKKKKKKKKRRRRRRRRCCCCCCCCCCCCJZZZZOOOOS
YKKKKKKKKKUUUUGBBBBBBBBPPBBBSSSSSUSSSQQKKKKKKKKKKKCCCCCCCCCCCCHHHYHYHHHHHHBBBBHHHHQQQQQQQQBBBBBBBKBKKKKKKKKKKKKRRRRRRRCCCCCCCCCCTCCZZZZZZZOS
KKKKKKKKKUUUUUUBBBBPPPPPPPSSSSSSSSSSSQKKKKKKKKKKKKCTCCCCCCCCYCHHHHHHHHBHHBBBBHHHHQQQQQQQQQQBBBBBBKBKKKKKKKKKKKRRRRRRRRRRCCCCCCCCTTZZZZZZZZSS
KKKKKKKKKKUUUUUBBBBPPPPPPPSSSSSSSSSSSQKKKKKKKKKKKKCCCCCCCCCYYYYYHHHHHHBBBBBBBGGGHQQQAQQQQQQQQBBBBBBKKKKKKEEEKKRRRRRRRRRRZCCCCCCCTTZZZZZZZZZZ
KKKKKKKUUUUUUUUUBBBPPPPPPPSSSSSSSSSSSQKOKKKKKKKKKKCCCCCCCCYYYYYYHHHHHHBBBBBBBGGGHHGGGQQQQQEEEEBBBBBBKKEKKEEEEERRRRRRRRRRCCTTCCCCTTTZZZZZZZZZ
EKKKKKKKKKUJUUUUBBBBPPPPPPSSSSSSSSSSSSSSKKKKKKKKKKCCCCCCCCYYYYYYHHHHHHBBBBBBUGGGHGGGGGQQQQEEEEEEEBBBBKEKEEEEEERRRRRLRRRRRRTTKKKKTZZZZZZZZZZU
KKKKKKKKKKUJUUAABJBBPPPHPPSSSSSSSSSSSSSKKKCKKKKKKKCCCRRRCCCYYYYYYHHHHBBBBBBBUGGGGGGGQQQQQQQEEEEEEBBBBEEEEEEEEEEERELLLLRRRRTTKKKKTTTTZZZZZZUU
GKKKKKKKKUUUUUUGGJJPPPPHHPSSSSSSSSSSSSSSSSCCCKKKKKCKRRRRRRYYYYYYHHHHHHBBBBBBBGGGGGGQQQQQQQEEEEEHHBBBEECCCEEEEEEEEELLLLRRLLTKKKKKTTTTZZZZZZUU
GKKKKKKKKUUGUGGGGJHHPPPHHPSHHSSSSSSSSSSSCCCCCKKKKKKKRRYYYYYYYYYYYHHHHBBBBBBBJGGGGGGGQQQQQEEEEEEHHBBBCCCCCCEEEEEEEELLLLRRQLTTKKKKTTTTZUZZZUUU
GGKKKKKKKAGGGGGGJJJJJJJJJJHHHHSSSSSSSSSSCCCCKKKKVKKRRRRRYYYYYYYYHHHHHHBBBBBBJGGGGGGGQQQHQQQQEEHHHCCCXCCCEEEEEEEEEELLLLLLLLTTTKKKTTTTZZZZZZZU
GGGKKKKKKGGXGGGGJJJJJJJJJJHHHVVVSSSSSSSSCCCCCCCVVVRRRRRRYOYVYCCHHHHHHHBBBBJJJGGGGGGGQHQHHHEEEHHHHCCCCCCCCEEEEEEELLLLLLLLLLTTTTTTTTTTTTTTZZZU
GGKKKKKKGGGGGGGGJJJJJJJJJJHHHVSSSSCSSSCCCCCCCCCVVVVRRVVVVVVVVHHHHHHHHHHBIBGGGGGGGGGGHHHHHHHEHHHHHCCCCCCCCCCCEEELLLLLLLLLLLLTTTTTTTTTTTTTTTZD
GGKKKKKKGGGGGGGGJJJJJJJJJJHHHVWQSCCCCCCCCCCCDCVVVVVVVVVVVVVVMHHHHHHHHHQBBBGQGGTGGGGGDHHHHHHHHHHHHCCCCCCCCCCEEEELLLLLLLLLLLLTTTTTTTTTTTTTTTTD
GGGKGGGGGGGGGGGGJJJJJJJJJJHSHWWWWCCCCCCCCCFFFFFVVVVVVVVVVVVVMHHMHMMMMMQQQQQQQQGGGGHHHHHHHHHHHHHHPPCCCCCCCCMQQEELLLLLLLLLLLTTTTTTTTTTTTTTTTTD
GGGGGGGGGGGGGGGGJJJJJJJJJJSSWWSWWCCCCCCCCDFFFFFVJVVVVVVVVVVVMHHMMMMMMQQQQQQQQQGGQQHHHHHHHHHHHHHHCCCCCCCCCCCQQQLLLLLLLLLLLLLTLKKKKTTTDDTDDDDD
UUUUGUGGGGGGGGGGJJJJJJJJJJSSSSSSCCCCGCGFFFFFFFFDVVVVVVVVVVVMMMMMMMMQQQQQQQQQQQQQQQHHHHHHHHHHHHCCCCCCCCCCCCCQQLLLLLLLLLLLLLLTLKKKKTTTDDDDDDDD
UUUUUUUGGGGGGGGGGGLLBBHMSSSSSSSSSGGGGGGFFFFFFFFDDDVVVVVVVVVMMMMMQQQQQQQQQQQQQQQQHHHHHHHHHHHHHHHCCCCCCCCCCCCQQLLLLLLLLLLLLLLLLKKKKTTTDDDDDDDD
UUUUUUUUGGGGGGGGGGLBBHHSSSSSSSSSGGGGGGGFFFFFFFFDDDVVVVVVVVMMMMMMMMQQQQQQQQQQQQQQHHHHHHHHHHHHHHHCCCCCCCCCCQQQLLLLLPLLLLLLLLLLLLKKKKTTTDDDDDDD
UUUUUUUUGGGGGSSSGLLLBSSSSSSSSSSSSGGGGGGFFFFFFFDDDDDVVVVVVVMMMMMMQQQQQQQQQQQQQQQPTTHHHHHHHHHHHHHCCCCCCCCCQQQQQQLBBPLLLAALLLLKKKKKKKKTTDDDDDDD
UUUUUUUUUUUGSSSSSBBBBBSSSSSSSSSSSRGGGGGFFFFFDDDDDDDVVVVVVVVMMMMMMMMQQQQQQQQQQQQPTTTHHHHHHHHHHHHCCCCCCCCQQQQQQQBBBBWLAAAALLLKKKKKKKTTDDDDDDDD
UUUUUUUUDUSGSSSSSSBBBBBSSSSSSSSSRRGGGGGFFFFFCDDDDDDVVVVVVCCMMMMMMMMMMQQQPPQPQPQPPTTHHHHHHHHHHHCCCCCCPCCQQQQQBBBBBBBAATTAATLKKKKKKKDDDDDDDDDD
UUUUUUUUUUSSSSSSSSBBBBSSSSSSSSSSRRGGGGGGFFFFCCDDDDDTTTVVVCCCCMMMMMMMMMQPPPPPPPPPPTHHHHHKKHHHHHHCCCCCQCQQQQQQBBBBBBBBATTTTTTTTKKKKNNNNDDDDDDD
UUUUUUUUUUSSSSSSSSBSSSSSSSSSSSSSSGGGGGGGGGCCCCCDDDDDDTVVVCCCCCMMMMMMMAATPPPPPPPPPTHHHHHKKHHHHHHCCCCQQQQQQQQBBBBBBBBBTTTTTTTTTKKKKKKNNNDDDDDD
UUUUUUUUSUSSSSSSSSSSSSSSSSSSSSSWGGGGGGGGGGCCCCCDDDDDDDDVVCCCCCMMMMMMMAAATPPPPPPPTTHKKKKKKHHHHHCCCQQQQQQQQQBBBBBBBBBTTOTTTTTTLKKKKKKNNNDDDDDD
UUUUUUSSSSSSSSSSSSSSSSSSSSSSSSSWPGGGGGGGGGCCCCCGDDIIIDHVVVCCCCCMMMMMAAAAPPPPPPPEEEKKKEKEEHHHHHHCQQQQQQQQQQQBBBBBBBZTTTTTTTTTKKKKEKKGGGGGGGDG
UUUUUUUUSSSSSSSSSSSSSSSUUSSSSWSWWGGGGCGGGGCCCCCCCIIIICCCCCCCCCCCJJMAAAAPPPPPPEEEEEEEEEEEEEEHHHHHHQQQQQQQQQQBBBBBBBZZTTTTTTTTTKKKKKKGGGGGGGGG
UUUUBUSUSSAAAASSSSSSSSSUUUSSSWWWWGGGGCGGCCCCCCCCCIIIIICCCCCCCCCCCCAAAAAAPPAAEEEEEEEEEEEEEEEHHHHHQQQCQQQQQQQBBBBBBBZZTTTTTTTTTTBKGGKGGHHGGGGG
DDUUUSSSSAAAAAAASSSSSSSSUUUWWEWVWGFGCCCCCCCCCCCCIIXXXICCCCCCCCCCAAAAAAAAPPPAPPPEEEEEEEEEEUUUUUHHMQQQQQQQQQQQBHBBBBBTTTTTTTTTTTTGGGGGHHHGHGGG
DUUUUSSSSSBBAAAASSSSSSSUUUWWWWWVWFFFFFCCCCCCCCCCXXXXXCCNCCCCCCCCAAAAAAAAPAAAPPEEEEEEEEEEEUEEEUUZZQQQQQQQQQNZHHHBBBBTTTTTTTTTTTTGGGGGHHHHHGGG
DDDDUDDSSSSAAAAAASSSSUUUUUWWWWWWWWFFFCCCCCCCCCCCXXXAXACCCCCCCCCCCAAAAAAAAAAAAPEEEEEEEEEEEEEEEUZZZZQQQQQQQQZZZHHHBBBSTTTTTTTTTTGGGGGHHHHHGGGG
DDDDDDDSSSZAAAAAASSSUUUUUUWWWWWWWWFFFFFFCCCCCCCCCXXAAACECCCCCCCCCCAAAAAAAAAAAAAEEEEEEEEEEEWZUUUZZZZQZQZQQQZHHHHHHHBTTTTTTTTTTTGGGGGHHGHGGGGG
DDDBDDDDSDAAAAAAAUUUUUUUUUWWWWWWWWFFFFCCCCCCCCCCCCAACCCCCCCCJCCCAAAAAAAAAAAAAAEEEEEEEEEEEEEZZUUZZZZZZZZQZZZHZHHHHHHHHTTTTTTTTGGGGGGHHGGGGGGG
DDDBBDDDSDDDDAAVVVUUUUUUUUUWWWFWWFFFFFCCCCCCCCCCCCCCCCCCCCCJATAAAAAAAAAAAAAAAAAAEEEEEEEEEEEZZZZZZZZZZZZZZZZZZHHHHHHHHTTTTTTTGGGGGGGHHGGGGGGG
DDDBBBDDDDDDDDAAAAAUUUUUUUUWFFFFFFFFFCCCCCCCCCCCCCCCCCJCCCCJAAAAAAAAAAAAAAAAAAAAEEEYYEEYYEEESSZZZZZZZZZZZZZZHHHHHHHHTTTTTTTGGGGGGGGGGGGGGGGG
DDDBBDDDDDDDDDDDAQQQUUUUUUUFFFFFFFFFFFFCCCCCCCCCCCCKCCJJJJJJAAAAAAAAAAAAAAAAAAAEEEYYYYYYYYESSSSSZZZZZZZZZZZZHHHHHHHHHTTTTTTGGGGGGGGGGGGGGGGG"""